In [25]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Create Landsat and Sentinel 2 Annual Composites

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/redcastle-resources/lcms-training/blob/main/2-Composites.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/redcastle-resources/lcms-training/blob/main/2-Composites.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/redcastle-resources/lcms-training/blob/main/2-Composites.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This notebook introduces the compositing methods used by the USDA Forest Service Landscape Change Monitoring System (LCMS). It then produces composites for a small LCMS study area. All composites are visualized using the `geeViz` Python package.

Learn more about [geeViz](link to geeViz doc!!).

The composites will be used to generate time series of data that the LCMS modeling algorithsm will use to predict areas of land use and land cover change, in the following modules. 

Learn more about [LCMS](https://apps.fs.usda.gov/lcms-viewer/home.html).

### Objective

This tutorial uses the following Google Cloud services:

- `Google Earth Engine`

The steps performed include:

- Testing various parameters to create composites
- Create composite asset

## Before you begin

### Set your current URL under `workbench_url`
* This will be in your URL/search bar at the top of the browser window you are currently in
* It will look something like `https://1234567890122-dot-us-west3.notebooks.googleusercontent.com/`

### Set a folder to use for all exports under `export_path_root` 
* It will be something like `projects/projectID/assets/someFolder`
* This folder does not have to already exist. If it does not exist, it will be created

In [27]:
workbench_url = 'https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/'
export_path_root  = 'projects/rcr-gee/assets/lcms-training'

print('Done')

Done


# Installation
Install necessary python packages. Uncomment the first line to upgrade geeViz if necessary.

In [32]:
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
# from IPython.display import IFrame,display, HTML
ee = getImagesLib.ee
Map = getImagesLib.Map

print('Done')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 685.7/685.7 kB 10.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: geeViz
    Found existing installation: geeViz 2023.7.5
    Uninstalling geeViz-2023.7.5:
      Successfully uninstalled geeViz-2023.7.5
Done


## Set up your work environment

Create a folder and collection where you will export the results of the compositing lab 

In [33]:
# Create folder and a collection and make public

export_composite_collection = f'{export_path_root}/lcms-training_module-2_composites'


aml.create_asset(export_composite_collection,asset_type = ee.data.ASSET_TYPE_IMAGE_COLL)

# Currently geeView within Colab uses a different project to authenticate through, so you may need to make your asset public to view from within Colab
aml.updateACL(export_composite_collection,writers = [],all_users_can_read = True,readers = [])



# aml.batchCopy('projects/rcr-gee/assets/composites-lcms-training-module-1',export_composite_collection,outType = 'imageCollection')
# # aml.batchDelete('projects/rcr-gee/assets/composites-lcms-training-module-1')
# print('Done')

AttributeError: module 'geeViz.assetManagerLib' has no attribute 'create_asset'

## Example 1: Default parameters

We'll get to know the parameters in the getImages Lib. 

For more documentation of the parameters in the getImages Lib, refer to this document: 
 -> link out to wrapper function in geeViz documentation and/or recreate here in Markdown format

In [34]:
#This example will use all default parameters to demonstrate how to use the basic composite functionality
Map.proxy_url = workbench_url

#First clear the map in case it has been populated with layers/commands earlier
Map.clearMap()

### Define parameters

The first parameters we will vary are: 
* ```studyArea``` - a polygon that defines the area we want to examine
* ```startJulian``` - 
* ```endJulian``` - 
* ```startYear``` - 
* ```endYear``` - 

In [35]:
# Define user parameters:

# Specify study area: Study area
# Can be a featureCollection, feature, or geometry
studyArea = ee.FeatureCollection('projects/lcms-292214/assets/R8/PR_USVI/Ancillary/prusvi_boundary_buff2mile')#.geometry().bounds(50,'EPSG:5070')
# studyArea = ee.Geometry.Polygon(
#         [[[-66.29465453845745, 18.491553939984392],
#           [-66.29465453845745, 18.144770192006572],
#           [-65.58054321033245, 18.144770192006572],
#           [-65.58054321033245, 18.491553939984392]]], None, False)

# Update the startJulian and endJulian variables to indicate your seasonal
# constraints. This supports wrapping for tropics and southern hemisphere.
# If using wrapping and the majority of the days occur in the second year, the system:time_start will default
# to June 1 of that year.Otherwise, all system:time_starts will default to June 1 of the given year
# startJulian: Starting Julian date
# endJulian: Ending Julian date
startJulian = 152
endJulian = 151

# Specify start and end years for all analyses
# More than a 3 year span should be provided for time series methods to work
# well. If providing pre-computed stats for cloudScore and TDOM, this does not
# matter
startYear = 2009
endYear = 2011

### Call on master wrapper function
link out to documentation about master wrapper function. 
Will print out all parameters below. 
Set verbose = FALSE (or whatever this is!!!) to make it quieter

In [36]:
#Call on master wrapper function to get Landat scenes and composites
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea.geometry().bounds(50,'EPSG:5070'),
                                                           startYear,
                                                           endYear,
                                                           startJulian,
                                                           endJulian)

#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

Get Processed Landsat and Sentinel2 Scenes: 
Start date: Jun 01 2009 , End date: May 30 2012
startYear :  2009
endYear :  2011
startJulian :  152
endJulian :  151
toaOrSR :  TOA
includeSLCOffL7 :  False
defringeL5 :  False
applyQABand :  False
applyCloudProbability :  True
applyShadowShift :  False
applyCloudScoreLandsat :  False
applyCloudScoreSentinel2 :  False
applyTDOMLandsat :  True
applyTDOMSentinel2 :  True
applyFmaskCloudMask :  True
applyFmaskCloudShadowMask :  True
applyFmaskSnowMask :  False
cloudScoreThresh :  20
performCloudScoreOffset :  True
cloudScorePctl :  10
zScoreThresh :  -1
shadowSumBands :  ['nir', 'swir1']
landsatResampleMethod :  near
sentinel2ResampleMethod :  aggregate
convertToDailyMosaics :  True
runChastainHarmonization :  True
correctIllumination :  False
correctScale :  250
preComputedLandsatCloudScoreOffset :  None
preComputedLandsatTDOMIRMean :  None
preComputedLandsatTDOMIRStdDev :  None
preComputedSentinel2CloudScoreOffset :  None
preComputedSentinel

### Add to Map
Describe geeViz map functions - basically, the same as default functions in GEE Playground. 

The map viewer will appear below.

In [37]:
Map.clearMap()

Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'Default Params {}-{}'.format(startJulian,endJulian),'True')
Map.addLayer(studyArea,{},'Study Area')
Map.centerObject(studyArea)
Map.turnOnInspector()
Map.view()

Adding layer: Default Params 152-151
Adding layer: Study Area
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1234/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3b40cb8a2076f1b6-dot-us-west3.notebooks.googleusercontent.com/proxy/1234/geeView/?accessToken=None


#### Inspect the outputs
#It is clear the default parameters do not work very well in this area
#There are missing data and cloud artifacts
#You can access the parameters that were used through the properties of the returned collection

In [ ]:
#It is clear the default parameters do not work very well in this area
#There are missing data and cloud artifacts
#You can access the parameters that were used through the properties of the returned collection
print(processedComposites.toDictionary().getInfo())
print(processedScenes.toDictionary().getInfo())



## Example 2: Include Landsat 7
Since there are not that many images available in this area for these years, let's try adding Landsat .

### Add a new parameter

We'll do this by adding a new parameter to the getImagesLib wrapper function: 
``` includeSLCoffL7 ``` 

In [25]:
#Since there are not that many images available in this area for these years, let's try adding Landsat 7
includeSLCOffL7 = True

### Call on wrapper function again, including new parameter

In [25]:
#Call on master wrapper function to get Landat scenes and composites
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea.geometry().bounds(50,'EPSG:5070'),startYear,endYear,startJulian,endJulian,includeSLCOffL7=includeSLCOffL7)


#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

### Add to map

In [25]:
#Turn off layers from previous iteration
Map.turnOffAllLayers()

# Map.addLayer(processedComposites.select(['NDVI','NBR']),{'addToLegend':'false'},'Time Series (NBR and NDVI)',False)
# for year in range(startYear,endYear + 1 ):
    #  t = processedComposites.filter(ee.Filter.calendarRange(year,year,'year')).mosaic()
Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'L7 added {}-{}'.format(startJulian,endJulian),True)


Map.view()

### Inspect the outputs
You'll notice this helps fill in the holes, but introduces many cloud-related artifacts

## Example 3: Improve Cloud masking 
Let's try to improve the cloud masking. Fmask is used by default, but misses some clouds
We'll try the adding in the cloudScore method
We do this by setting the ```applyCloudScore``` parameter
To learn more about the cloudScore cloud masking method, visit: 

### Add cloud masking parameter

In [26]:
#Let's try to improve the cloud masking. Fmask is used by default, but misses some clouds
#We'll try the adding in the cloudScore method
applyCloudScore = True

### Run master wrapper function
Keeping all other parameters the same

In [26]:
#Call on master wrapper function to get Landat scenes and composites
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea.geometry().bounds(50,'EPSG:5070'),
                                                           startYear,
                                                           endYear,
                                                           startJulian,
                                                           endJulian,
                                                           includeSLCOffL7=includeSLCOffL7,
                                                           applyCloudScoreLandsat=applyCloudScore)


#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

### Add to Map

In [26]:
#Turn off layers from previous iteration
Map.turnOffAllLayers()


Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'L7 and CloudScore added {}-{}'.format(startJulian,endJulian),True)


Map.view()

### Inspect
You'll notice this cleans up the cloud masking a lot
You'll still notice there are some dark areas likely due to cloud shadow masking omission
Fmasks's cloud shadow mask misses a lot typically. A temporal outlier method called the
Temporal Dark Outlier Mask (TDOM) works well with masking cloud shadows. 



## Example 4: Use Temporal Dark Outlier Mask (TDOM) to Mask shadows
One sentence summary of TDOM + Link to documentation about TDOM

### Set TDOM parameter
``` applyTDOM ```

In [27]:
#We'll try the TDOMe method
applyTDOM = True

### Run master wrapper function

In [27]:
#Call on master wrapper function to get Landat scenes and composites
#In order to identify dark outliers, we will extend the dates by 6 years to get a larger sample
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(studyArea.geometry().bounds(50,'EPSG:5070'),
                                                           startYear,
                                                           endYear,
                                                           startJulian,
                                                           endJulian,
                                                           includeSLCOffL7=includeSLCOffL7,
                                                           applyCloudScoreLandsat=applyCloudScore,
                                                           applyTDOM = applyTDOM)


#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

### Add to map

In [27]:
#Turn off layers from previous iteration
Map.turnOffAllLayers()

Map.addTimeLapse(processedComposites,getImagesLib.vizParamsFalse,'CloudScore and TDOM added {}-{}'.format(startJulian,endJulian),True)


Map.view()

### Inspect
You'll notice this cleans up the cloud masking a lot. 

### TDOM is computationally intensive
However, while TDOM is a great cloud shadow masking method, it is a bit computationally intensive since it computes the mean and standard deviation over a large stack of data.

In order to avoid re-computing the stats, we store pre-computed stats. For the LCMS project, we have precomputed stats for CONUS, AK, HI, PuertoRico, and the USVI. 
(do we want to point people to these stats? )

It is optional that you use pre-computed stats. If none are provided, TDOM stats will be computed on-the-fly. 

You can create and store precomputed TDOM stats using the following code chunk: 

```
{
  this is an example set of code to show people how to compute TDOM stats
  we'll fill this in later!!
}
```
### Inspect TDOM stats

We can look at just the raw TDOM stats on the map. 

In [11]:
# While TDOM is a great cloud shadow masking method, it is a bit computationally intensive since it computes the mean and standard deviation over a large stack of data
# In order to avoid re-computing the stats, we store pre-computed stats. It is optional that you use pre-computed stats. If none are provided, TDOM stats will be computed on-the-fly

Map.clearMap()

landsat_tdom_stats = ee.Image('projects/lcms-tcc-shared/assets/CS-TDOM-Stats/PR-USVI/TDOM_stats/Landsat_TDOM_Stats_1984_2021')\
                    .select(['Landsat_nir_.*','Landsat_swir1_.*'])\
                    .divide(10000)
s2_tdom_stats = ee.Image('projects/lcms-tcc-shared/assets/CS-TDOM-Stats/PR-USVI/TDOM_stats/Sentinel2_TDOM_Stats_2015_2021')\
                    .select(['Sentinel2_nir_.*','Sentinel2_swir1_.*'])\
                    .divide(10000)
Map.addLayer(landsat_tdom_stats,{'min':0.15,'max':0.35,'bands':'Landsat_nir_mean','palette':'222,080'},'Landsat TDOM Stats')
Map.addLayer(s2_tdom_stats,{'min':0.15,'max':0.35,'bands':'Sentinel2_nir_mean','palette':'222,080'},'Sentinel 2 TDOM Stats')

Map.turnOnInspector()
Map.view()




Adding layer: Landsat TDOM Stats
Adding layer: Sentinel 2 TDOM Stats
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


127.0.0.1 - - [03/Aug/2023 21:19:43] "GET /geeView/?accessToken=None HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:19:43] "GET /geeView/js/lcms-viewer.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:19:43] "GET /geeView/js/load.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:19:43] "GET /geeView/js/gena-gee-palettes.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:19:43] "GET /geeView/css/style.min.css HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:19:43] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:19:43] "GET /geeView/images/GEE_logo_transparent.png HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:19:43] "GET /geeView/images/logos_usda-fs.svg HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:19:43] "GET /geeView/images/usfslogo.png HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:19:43] "GET /geeView/images/GEE.png HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:19:43] "GET /geeView/images/menu-hamburger_ffffff.svg HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/202

#Define user parameters:

# Specify study area: Study area
# Can be a featureCollection, feature, or geometry
studyArea = getImagesLib.testAreas['CA']

# Update the startJulian and endJulian variables to indicate your seasonal 
# constraints. This supports wrapping for tropics and southern hemisphere.
# If using wrapping and the majority of the days occur in the second year, the system:time_start will default 
# to June 1 of that year.Otherwise, all system:time_starts will default to June 1 of the given year
# startJulian: Starting Julian date 
# endJulian: Ending Julian date
startJulian = 152
endJulian = 273

# Specify start and end years for all analyses
# More than a 3 year span should be provided for time series methods to work 
# well. If providing pre-computed stats for cloudScore and TDOM, this does not 
# matter
startYear = 2018
endYear = 2022

# Specify an annual buffer to include imagery from the same season 
# timeframe from the prior and following year. timeBuffer = 1 will result 
# in a 3 year moving window. If you want single-year composites, set to 0
timebuffer =0

# Specify the weights to be used for the moving window created by timeBuffer
# For example- if timeBuffer is 1, that is a 3 year moving window
# If the center year is 2000, then the years are 1999,2000, and 2001
# In order to overweight the center year, you could specify the weights as
# [1,5,1] which would duplicate the center year 5 times and increase its weight for
# the compositing method. If timeBuffer = 0, set to [1]
weights = [1]

# Choose medoid or median compositing method. 
# Median tends to be smoother, while medoid retains 
# single date of observation across all bands
# The date of each pixel is stored if medoid is used. This is not done for median
# If not exporting indices with composites to save space, medoid should be used
compositingMethod = 'medoid'

# Choose which Landsat USGS Collection version to use
# Choices are 'C1' for Collection 1 and 'C2' for Collection 2
# Only choose 'C1' if working with other Collection 1 data and using
# data from before 1/1/2022. Otherwise, choose 'C2'
# See: https://www.usgs.gov/landsat-missions/landsat-collection-2 for more information
landsatCollectionVersion = 'C2'

# Choose Top of Atmospheric (TOA) or Surface Reflectance (SR)
# Use caution when combining Landsat and S2 SR data since S2 SR data had terrain correction performed 
toaOrSR = 'TOA'

# Whether to convert S2 images from the military grid reference system(MGRS) tiles to daily mosaics to avoid arbitrary
# MGRS tile artifacts or not. In most cases, it is best to set this to true.
convertToDailyMosaics = True


# Choose whether to include Landat 7
# Generally only included when data are limited
includeSLCOffL7 = True

# Whether to defringe L4 and L5
# Landsat 4 and 5 data have fringes on the edges that can introduce anomalies into 
# the analysis.  This method removes them, but is somewhat computationally expensive
defringeL5 = True

# Choose cloud/cloud shadow masking method
#  Choices are a series of booleans for cloudScore, TDOM, and elements of Fmask
# Fmask masking options will run fastest since they're precomputed
# Fmask cloud mask is generally very good, while the fMask cloud shadow
# mask isn't great. TDOM tends to perform better than the Fmask cloud shadow mask. cloudScore 
# is usually about as good as the Fmask cloud mask overall, but each failes in different instances.
# CloudScore runs pretty quickly, but does look at the time series to find areas that 
# always have a high cloudScore to reduce commission errors- this takes some time
# and needs a longer time series (>5 years or so)
# TDOM also looks at the time series and will need a longer time series
# If pre-computed cloudScore offsets and/or TDOM stats are provided below, cloudScore
# and TDOM will run quite quickly

#CloudScore and TDOM switches- for both Sentinel 2 and Landsat separately
#We generally use cloudScore for Landsat, but not Sentinel 2
applyCloudScoreLandsat = True
applyCloudScoreSentinel2 = False

applyTDOMLandsat = True
applyTDOMSentinel2 = True

#S2 only cloud/cloud shadow masking methods switches- generally do not use these
#QA band method is fast but is generally awful- don't use if you like good composites
#Shadow shift is intended if you don't have a time series to use for TDOM or just want individual images
#It will commit any dark area that the cloud mask is cast over (water, hill shadows, etc)
applyQABand = False
applyShadowShift = False
#Height of clouds to use to project cloud shadows
cloudHeights = ee.List.sequence(500,10000,500)

#Whether to use the pre-computed cloud probabilities to mask
#clouds for Sentinel 2
#This method works really well
applyCloudProbability = True

#If cloudProbability is chosen, choose a threshold 
#(generally somewhere around 40-60 works well)
cloudProbThresh = 40

#Fmask switches- only for Landsat
#Generally we do use these
applyFmaskCloudMask = True
applyFmaskCloudShadowMask = True
applyFmaskSnowMask = False

# Cloud and cloud shadow masking parameters.
# If cloudScore  is chosen
# cloudScoreThresh: If using the cloudScore or cloudProbability method-Threshold for cloud 
#    masking (lower number masks more clouds.  Between 10 and 30 generally 
#    works best)
cloudScoreThresh = 20

#Whether to find if an area typically has a high cloudScore
#If an area is always cloudy, this will result in cloud masking omission
#For bright areas that may always have a high cloudScore
#but not actually be cloudy, this will result in a reduction of commission errors
#This procedure needs at least 5 years of data to work well
performCloudScoreOffset = True

# If performCloudScoreOffset = true:
#Percentile of cloud score to pull from time series to represent a minimum for 
# the cloud score over time for a given pixel. Reduces comission errors over 
# cool bright surfaces. Generally between 5 and 10 works well. 0 generally is a
# bit noisy but may be necessary in persistently cloudy areas
cloudScorePctl = 10

# zScoreThresh: Threshold for cloud shadow masking- lower number masks out 
#    less. Between -0.8 and -1.2 generally works well
zScoreThresh = -1

# shadowSumThresh: Sum of IR bands to include as shadows within TDOM and the 
#    shadow shift method (lower number masks out less)
shadowSumThresh = 0.35

# contractPixels: The radius of the number of pixels to contract (negative 
#    buffer) clouds and cloud shadows by. Intended to eliminate smaller cloud 
#    patches that are likely errors
# (1.5 results in a -1 pixel buffer)(0.5 results in a -0 pixel buffer)
# (1.5 or 2.5 generally is sufficient)
contractPixels = 1.5

# dilatePixels: The radius of the number of pixels to dilate (buffer) clouds 
#    and cloud shadows by. Intended to include edges of clouds/cloud shadows 
#    that are often missed
# (1.5 results in a 1 pixel buffer)(0.5 results in a 0 pixel buffer)
# (2.5 or 3.5 generally is sufficient)
dilatePixels = 2.5

# Choose the resampling method: 'aggregate','near', 'bilinear', or 'bicubic'
# Defaults to 'aggregate' for Sentinel 2 and 'near' for Landsat

# Aggregate is generally useful for aggregating pixels when reprojecting instead of resampling
# A good example would be reprojecting S2 data to 30 m

# If method other than 'near' is chosen, any map drawn on the fly that is not
# reprojected, will appear blurred or not really represented properly
# Use .reproject to view the actual resulting image (this will slow it down)
landsatResampleMethod = 'near'

sentinel2ResampleMethod = 'aggregate'

# Choose whether to use the Chastain et al 2019(https://www.sciencedirect.com/science/article/pii/S0034425718305212)
# harmonization method
# All harmonization models apply a rather small correction and are likely not needed
#Only runs if toaOrSR = 'TOA'
runChastainHarmonization = True

# If available, bring in preComputed cloudScore offsets and TDOM stats
# Set to null if computing on-the-fly is wanted
# These have been pre-computed for all CONUS for Landsat and Setinel 2 (separately)
# and are appropriate to use for any time period within the growing season
# The cloudScore offset is generally some lower percentile of cloudScores on a pixel-wise basis
preComputedCloudScoreOffset = getImagesLib.getPrecomputedCloudScoreOffsets(cloudScorePctl)
preComputedLandsatCloudScoreOffset = preComputedCloudScoreOffset['landsat']
preComputedSentinel2CloudScoreOffset = preComputedCloudScoreOffset['sentinel2']

# The TDOM stats are the mean and standard deviations of the two bands used in TDOM
# By default, TDOM uses the nir and swir1 bands
preComputedTDOMStats = getImagesLib.getPrecomputedTDOMStats()
preComputedLandsatTDOMIRMean = preComputedTDOMStats['landsat']['mean']
preComputedLandsatTDOMIRStdDev = preComputedTDOMStats['landsat']['stdDev']

preComputedSentinel2TDOMIRMean = preComputedTDOMStats['sentinel2']['mean']
preComputedSentinel2TDOMIRStdDev = preComputedTDOMStats['sentinel2']['stdDev']


# Export params
# Whether to export composites
exportComposites = False

#Set up Names for the export
outputName = 'Landsat_Sentinel2_Hybrid'

# Provide location composites will be exported to
# This should be an asset folder, or more ideally, an asset imageCollection
exportPathRoot = 'users/username/someCollection'



# CRS- must be provided.  
# Common crs codes: Web mercator is EPSG:4326, USGS Albers is EPSG:5070, 
# WGS84 UTM N hemisphere is EPSG:326+ zone number (zone 12 N would be EPSG:32612) and S hemisphere is EPSG:327+ zone number
crs = 'EPSG:5070'

# Specify transform if scale is null and snapping to known grid is needed
transform = [10,0,-2361915.0,0,-10,3177735.0]

# Specify scale if transform is null
scale = None

#### Example 4 - Prepare final composites and export
Now that we've gone through some of the parameters that can be changed for Landsat composites, we will use both Landsat and Sentinel2 to get the best possible composites. 

Here, we're manually setting all parameters. 

Review the documentation [link] to recall the default values for each of these parameters.

We are also using parameters in the function to export our composites to the root path we specified earlier. 

```
    {
      exportComposites = True,
      outputName = 'Landsat-Sentinel2-Hybrid',
      exportPathRoot = export_composite_collection,
    }
```

In [12]:
# Run get images, set parameters manually, and export
lsAndTs = getImagesLib.getLandsatAndSentinel2HybridWrapper(\
  studyArea = studyArea,
  startYear = 1984,
  endYear = 2022,
  startJulian = 152,
  endJulian = 151,
  timebuffer =  0,
  weights =  [1],
  compositingMethod = 'medoid',
  toaOrSR = 'TOA',
  includeSLCOffL7 = True,
  defringeL5 = True,
  applyQABand = False,
  applyCloudProbability = True,
  applyShadowShift = False,
  applyCloudScoreLandsat = True,
  applyCloudScoreSentinel2 = False,
  applyTDOMLandsat = True,
  applyTDOMSentinel2 = True,
  applyFmaskCloudMask = True,
  applyFmaskCloudShadowMask = True,
  applyFmaskSnowMask = False,
  cloudHeights = ee.List.sequence(500,10000,500),
  cloudScoreThresh = 10,
  performCloudScoreOffset = False,
  cloudScorePctl = 10,
  zScoreThresh = -1,
  shadowSumThresh = 0.35,
  contractPixels = 1.5,
  dilatePixels = 3.5,
  shadowSumBands = ['nir','swir1'],
  landsatResampleMethod = 'bicubic',
  sentinel2ResampleMethod = 'bicubic',
  convertToDailyMosaics = True,
  runChastainHarmonization = False,# Set to True in order to perform regression harmonization between Landsat and Sentinel 2
  correctIllumination = False,
  correctScale = 250,
  exportComposites = True,
  outputName = 'Landsat-Sentinel2-Hybrid',
  exportPathRoot = export_composite_collection,
  crs = getImagesLib.common_projections['NLCD_CONUS']['crs'],
  transform = getImagesLib.common_projections['NLCD_CONUS']['transform'],
  scale = None,
  preComputedLandsatCloudScoreOffset = None,
  preComputedLandsatTDOMIRMean = landsat_tdom_stats.select(['.*_mean']),
  preComputedLandsatTDOMIRStdDev = landsat_tdom_stats.select(['.*_stdDev']),
  preComputedSentinel2CloudScoreOffset = None,
  preComputedSentinel2TDOMIRMean = s2_tdom_stats.select(['.*_mean']),
  preComputedSentinel2TDOMIRStdDev = s2_tdom_stats.select(['.*_stdDev']),
  cloudProbThresh = 40,
  landsatCollectionVersion = 'C2',
  overwrite = False)



Get Processed Landsat and Sentinel2 Scenes: 
Start date: May 31 1984 , End date: May 31 2023
Get Processed Landsat: 
Start date: May 31 1984 , End date: May 31 2023
Defringing L4 and L5
Including All Landsat 7
Setting resample method to  bicubic
Applying Cloud Score
Not computing cloudScore offset
Applying Fmask Cloud Mask
Applying TDOM Shadow Mask
Using pre-computed irMean for TDOM
Using pre-computed irStdDev for TDOM
Applying Fmask Shadow Mask
Get Processed Sentinel2: 
Start date: May 31 1984 , End date: May 31 2023
Using S2 Collection: COPERNICUS/S2_HARMONIZED
Joining pre-computed cloud probabilities from: COPERNICUS/S2_CLOUD_PROBABILITY
Setting resample method to  bicubic
Converting S2 data to daily mosaics
Apply Cloud Probability
Applying TDOM
Using pre-computed irMean for TDOM
Using pre-computed irStdDev for TDOM
pyramiding object: {'.default': 'mean'}
Exporting: Landsat-Sentinel2-Hybrid_TOA_medoid_1984_1984_152_151
<Task EXPORT_IMAGE: Landsat-Sentinel2-Hybrid_TOA_medoid_1984_198

127.0.0.1 - - [03/Aug/2023 21:23:36] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:23:53] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:24:07] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:25:21] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:25:45] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:27:23] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:28:00] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:28:22] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:32:16] "GET /geeView/?accessToken=None HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:32:16] "GET /geeView/css/style.min.css HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:32:16] "GET /geeView/js/lcms-viewer.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Aug/2023 21:32:16] "GET /geeView/js/load.min.js HTTP/1.1" 2

### Track export tasks

In [9]:
# If you'd like to track the tasks, use this:
# tml.trackTasks2()

# If you want to cancel all running tasks, you can use this function
# tml.batchCancel()

# If you want to empty the collection of all images
# aml.batchDelete(export_composite_collection, type = 'imageCollection')

print('done')

done


### View composites as they are completed

In [39]:
# View composites as they are completed
composites = ee.ImageCollection(export_composite_collection)
Map.clearMap()

### Inspect

In [13]:
# First let's explore the composites and some attributes that can help understand how well the composites turned out
Map.addTimeLapse(composites,getImagesLib.vizParamsFalse10k,'Composites')

# By looking at the Sensor that is used, you can see 
# with the introduction of Landsat 8 in 2013 and then Sentinel 2 in 2016-2017, 
# composites become much better quality
Map.addTimeLapse(composites.select('sensor'),{'min':4,'max':22,'palette':'088,808','classLegendDict':{'Landsat (Landsat 5 = 5, Landsat 7 = 7, etc...)':'088','Sentinel2 (Sentinel 2A = 21, Sentinel 2B = 22, etc....)':'808'}},'Sensor')
Map.addTimeLapse(composites.select('compositeObsCount'),{'min':3,'max':10,'palette':'D0D,0D0'},'Composite Observation Counts')
Map.turnOnInspector()
Map.view()
# Now that we have exported composites, we will use them in the LandTrendr temporal segmentation algorithm

Adding layer: Composites
Adding layer: Sensor
Adding layer: Composite Observation Counts
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


127.0.0.1 - - [04/Aug/2023 03:16:41] "GET /geeView/?accessToken=None HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2023 03:16:41] "GET /geeView/js/lcms-viewer.min.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2023 03:16:42] "GET /geeView/css/style.min.css HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2023 03:16:42] "GET /geeView/js/load.min.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2023 03:16:42] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2023 03:16:42] "GET /geeView/js/gena-gee-palettes.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2023 03:16:42] "GET /geeView/images/GEE.png HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2023 03:16:42] "GET /geeView/images/GEE_logo_transparent.png HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2023 03:16:42] "GET /geeView/images/logos_usda-fs.svg HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2023 03:16:42] "GET /geeView/images/usfslogo.png HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2023 03:16:42] "GET /geeView/images/usdalogo.png HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2023 03:16:42] "